In [1]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime
from dotenv import load_dotenv
import os

In [2]:
download_path = os.path.join(os.path.expanduser("~"), "Downloads")

In [3]:
file_path = rf"{download_path}\Base Consolidada 2025.xlsx"

In [4]:
email_remetente = "projetos@polijunior.com.br"
load_dotenv(dotenv_path=".venv/.env")
senha = "projetos.pj"

In [5]:
def send_email(remetente, destinatario, assunto, corpo, senha):
    try:
        # Criação da mensagem
        msg = MIMEMultipart()
        msg["From"] = remetente
        msg["To"] = destinatario
        msg["Subject"] = assunto

        # Corpo do e-mail
        msg.attach(MIMEText(corpo, "html", "utf-8"))

        # Conectar ao servidor SMTP do Gmail
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()  # Encriptação TLS

        # Login no servidor
        server.login(remetente, senha)

        # Envio do e-mail
        text = msg.as_string()
        server.sendmail(remetente, destinatario, text)

        # Encerrar a conexão
        server.quit()

        print(f"E-mail enviado com sucesso para {destinatario}")
    except Exception as e:
        print(f"Falha ao enviar e-mail para {destinatario}: {e}")

In [6]:
def send_bulk_emails(remetente, destinatarios, assunto, corpo, senha):
    """Envia e-mails em massa para uma lista de destinatários usando uma única conexão SMTP."""
    try:
        # Criação da mensagem
        msg = MIMEMultipart()
        msg["From"] = remetente
        msg["Subject"] = assunto
        msg.attach(MIMEText(corpo, "html", "utf-8"))
        text = msg.as_string()

        # Conectar ao servidor SMTP do Gmail
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()  # Encriptação TLS

        # Login no servidor
        server.login(remetente, senha)

        # Envio dos e-mails em massa
        for destinatario in destinatarios:
            try:
                server.sendmail(remetente, destinatario, text)
                print(f"E-mail enviado com sucesso para {destinatario}")
            except Exception as e:
                print(f"Falha ao enviar e-mail para {destinatario}: {e}")

        # Encerrar a conexão
        server.quit()

        print("Envio em massa concluído.")

    except Exception as e:
        print(f"Falha no envio em massa: {e}")

# 

# Feeds quinzenais

In [8]:
# Lemos a sheet "Base" da base consolidada
base_consolidada = pd.read_excel(file_path, sheet_name="Base")

In [9]:
# Jogando linhas nulas na coluna 'Projeto', resetando os índices da base e filtrando para projetos em execução
base_consolidada_exec = base_consolidada.dropna(subset=["Projeto"]).reset_index(
    drop=True
)
base_consolidada_exec = base_consolidada_exec[
    base_consolidada_exec["Status"] == "Executando"
].copy()

# Convertendo 'Início Real' para datetime e filtrando para projetos após a data de início da coleta do CSAT
base_consolidada_exec["Início Real"] = pd.to_datetime(
    base_consolidada_exec["Início Real"], dayfirst=True, errors="coerce"
)
base_consolidade_exec_atual = base_consolidada_exec[
    base_consolidada_exec["Início Real"] > pd.Timestamp("2024-07-03")
]

In [10]:
assunto = "[LEMBRETE]: Preenchimento do Formulário Quinzenal de Projetos"

corpo = """<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
</head>
<body>
    <p>Bom dia, equipe!</p>

    <p>Gostaria de lembrá-los de preencher o <a href="https://www.google.com/url?q=https://forms.gle/HCM1DeVVt8EdELRf9&sa=D&source=calendar&ust=1729946166398169&usg=AOvVaw1tuZHpJl2Fl-oZVaZIwjr5">formulário quinzenal de projetos</a>. 
    Ele é essencial para acompanharmos o andamento dos nossos projetos e garantirmos a evolução contínua do nosso trabalho.</p>

    <p>É fundamental que todos participem desse processo para que possamos construir um ambiente colaborativo e sempre em busca de melhorias. Contamos com a colaboração de todos para garantir que o formulário seja preenchido com atenção e pontualidade.</p>

    <p>Se tiverem alguma dúvida ou precisarem de ajuda, não hesitem em entrar em contato!</p>

    <p>Obrigado pela compreensão e bora buscar mais ENBs! 🐺</p>
</body>
</html>"""

In [11]:
# Adquirindo a lista de validadores atuais
validadores = base_consolidade_exec_atual[
    base_consolidade_exec_atual["Validador/Coordenador"].notna()
]["Validador/Coordenador"].unique()

In [12]:
lista_emails = []

In [13]:
for validador in validadores:
    lista_emails.append(validador + "@polijunior.com.br")
    print(f"E-mail preparado para {validador}")

E-mail preparado para antoniel.neves
E-mail preparado para kamila.freitas 
E-mail preparado para gabriele.shimizu
E-mail preparado para pedro.gil
E-mail preparado para guilherme.nunes
E-mail preparado para joao.cardoso
E-mail preparado para ayana.tanaka
E-mail preparado para mateus.ono
E-mail preparado para mateus.araújo
E-mail preparado para carlos.tavares
E-mail preparado para caio.castro
E-mail preparado para gustavo.loiola
E-mail preparado para rafael.maesano
E-mail preparado para diego.nozawa
E-mail preparado para matheus.santos
E-mail preparado para enzo.giannoni
E-mail preparado para guilherme.boen
E-mail preparado para natan.palacio
E-mail preparado para joao.melo
E-mail preparado para gustavo.timbo


# TESTE

In [14]:
raise KeyboardInterrupt

KeyboardInterrupt: 

In [15]:
email_teste = "gabriel.agra@polijunior.com.br"

In [16]:
# Mandamos 1 e-mail de teste
if lista_emails:
    email = lista_emails[0]
    send_email(
        remetente=email_remetente,
        destinatario=email_teste,  # Manda para email de teste
        assunto=f"[TESTE - {email}]: {assunto}",
        corpo=corpo,
        senha=senha,
    )

Falha ao enviar e-mail para gabriel.agra@polijunior.com.br: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 98e67ed59e1d1-309f7763f8csm2108622a91.26 - gsmtp')


# E-MAIL EM MASSA

In [ ]:
raise KeyboardInterrupt

In [ ]:
send_bulk_emails(
    remetente=email_remetente,
    destinatarios=lista_emails,  # Manda para lista de emails dos analistas
    assunto=assunto,
    corpo=corpo,
    senha=senha,
)